In [1]:
# !pip install transformers==4.46.2 datasets==3.1.0 numpy==1.26.4 sklearn-pandas==2.2.0 torch==2.5.1+cu121

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
# from sklearn.metrics import f1_score, precision_score, recall_score
import pandas as pd
import numpy as np
import datasets
import time
import wandb
import random
# import os
# from fvcore.nn import FlopCountAnalysis
# from torch.utils.data import DataLoader
import torch

key_file = r'C:\Development\TactitalTensorsFinalProject\WANDB_API_KEY.txt' 

with open(key_file, "r") as f:
    api_key = f.read().strip()

# Log into WandB with the API key
wandb.login(key=api_key)

# Initialize WandB (no need to manually set the WANDB_API_KEY env variable again)


c:\Users\ayoun\Miniconda3\envs\nbse2025\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ayoun\Miniconda3\envs\nbse2025\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\ayoun\Miniconda3\envs\nbse2025\lib\site-packages\torc

True

The WANDB_API_KEY is nessesary to use the Trainer class from HuggingFace and is what's used in the example notebook. However, I think we may all have to put in our own api key secret into google colab for it to work.

In [3]:
# from google.colab import userdata
# os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')


In [4]:
langs = ['java', 'python', 'pharo']

# langs = ['java'] # Using Java as the only language for testing purposes.
# langs = ['python']
# langs = ['pharo']
labels = {
    'java': ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational'],
    'python': ['Usage', 'Parameters', 'DevelopmentNotes', 'Expand', 'Summary'],
    'pharo': ['Keyimplementationpoints', 'Example', 'Responsibilities', 'Classreferences', 'Intent', 'Keymessages', 'Collaborators']
}
ds = datasets.load_dataset('NLBSE/nlbse25-code-comment-classification')

In [5]:
ds

DatasetDict({
    java_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 7614
    })
    java_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1725
    })
    python_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1884
    })
    python_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 406
    })
    pharo_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1298
    })
    pharo_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 289
    })
})

In [6]:
ds['java_test'][0]

{'index': 5,
 'class': 'AbstractContractGetFileStatusTest.java',
 'comment_sentence': 'accept everything.',
 'partition': 1,
 'combo': 'accept everything. | AbstractContractGetFileStatusTest.java',
 'labels': [0, 0, 1, 0, 0, 0, 0]}

In [7]:
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

The next few functions are to preprocess the trainng and validation sets, and to let the Trainer class to evaluate how well the trainng is going after each epoch.

In [8]:
# To tokenize the text in the 'combo' column of the training dataset of each language.

def tokenize_dataset(examples):
  return tokenizer(examples['combo'], truncation=True, padding="max_length", max_length=128)

In [9]:
# Tokenizes the text 'combo' column of the dataset, changes the values of the
# labels column to float instead of int, and sets to format of the lables column
# to torch tesnsors, which is required by the Trainer class.

def preprocess_dataset(input_dataset):
  processed_dataset = input_dataset.map(tokenize_dataset, batched=True, load_from_cache_file=False)
  processed_dataset = processed_dataset.cast_column("labels", datasets.features.Sequence(datasets.features.Value("float32")))
  processed_dataset.set_format('pt')
  return processed_dataset

Evaluating the predictions.

Below is non-functional code I still haven't finnished. We need to implement a way for the logits returned from the model to be converted into binary (0 - 1) predictions for the labels, and for those labels to be compaired to the ground truth labels of the testing datasets for each language. The competition also wants us to complete this step using profiling that they have set up, and which is partially shown below. The link to the reference notebook for the competition is here. Also, another notebook that is fairly close to what I believe we are trying to do can be found here_for_multi_label_text_classification.ipynb).

In [10]:
# scores must be global so that it can later be referenced for printing the results
scores = pd.DataFrame(columns=['model', 'lan', 'cat', 'precision', 'recall', 'f1'])

def compute_metrics(eval_pred, lang, categories, mod_name):
    global scores
    #scores.drop(scores[scores['lan'] == lang].index, inplace=True)

    predictions, true_labels = eval_pred
    predictions = (predictions > 0.5).astype(int)
    num_classes = len(categories)

    metrics_list = []

    for i in range(num_classes):
        y_pred = predictions[:, i]
        y_true = true_labels[:, i]

        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        metrics_list.append({
            'model': mod_name,
            'lan': lang,
            'cat': categories[i],
            'precision': precision,
            'recall': recall,
            'f1': f1
        })

    temp_scores = pd.DataFrame([m for m in metrics_list if m['model'] == mod_name])
    scores = pd.concat([scores, temp_scores], ignore_index=True)

    avg_f1 = temp_scores['f1'].mean()
    avg_precision = temp_scores['precision'].mean()
    avg_recall = temp_scores['recall'].mean()

    return {'f1': avg_f1, 'precision': avg_precision, 'recall': avg_recall}

In [11]:
def measure_runtime_and_flops(total_time, total_flops, trainer, validation_dataset):

    for batch in validation_dataset:
        inputs = {key: val.unsqueeze(0).to(trainer.model.device) for key, val in batch.items() if key in ['input_ids', 'attention_mask']}
        with torch.profiler.profile(with_flops=True, activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA]) as prof:
            start_time = time.time()
            _ = trainer.model(**inputs)
            end_time = time.time()

        total_time += (end_time - start_time)
        total_flops += sum(k.flops for k in prof.key_averages()) / 1e9 # Convert Flops to GFLOPs

    avg_runtime = total_time / len(validation_dataset)
    avg_flops = (total_flops / len(validation_dataset))
    
    return avg_runtime, avg_flops, total_flops, total_time

In [12]:
def compute_metrics_wrapper(mod_name):
    def inner(eval_pred):
        return compute_metrics(eval_pred, lang, labels[lang], mod_name)
    return inner


In [13]:
max_avg_runtime = 5
max_avg_flops = 5000

def score(avg_f1, avg_runtime, avg_flops):
    return (0.6 * avg_f1 +
            0.2 * ((max_avg_runtime - avg_runtime) / max_avg_runtime) +
            0.2 * ((max_avg_flops - avg_flops) / max_avg_flops))

In [14]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # for CUDA devices
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False  # Disable to ensure deterministic operations

In [15]:
# Training the models for each language and saving the the models and thier
# tokenizers after training.

# trying different pre-trained models such as roberta-base, distilbert-base, or codebert-large

seed = 27
set_seed(seed)

# learning_rates = [5e-5, 5e-7]
lr = 5e-5
epochs = [5, 10, 15, 20]
batch_sizes = [4, 8]
weights_of_decay = [0.01] #, 0.001]
final_model_stats = {}
lang_model_stats = {}


for epoch in epochs:
  for bs in batch_sizes:
    for wd in weights_of_decay:
      print(f'------------------ Starting model ==> epochs: {epoch}, batch size: {bs}, weights of decay: {wd} ---------------------')
      fin_mod_name = f'epoch-{epoch}_batchSize-{bs}_weightsOfDecay-{wd}'
      total_flops = 0
      total_time = 0
      total_avg_runtime = 0
      total_avg_flops = 0

      for lang in langs:

        lang_mod_name = f'{lang}_epoch-{epoch}_batchSize-{bs}_weightsOfDecay-{wd}'
        wandb.init(
            project="NBSE2025GridSearch2", 
            entity="ayoungren-colostate",
            name=lang_mod_name,
            config={  # Log hyperparameters for each run
                "epochs": epoch,
                "batch_size": bs,
                "weight_decay": wd,
                "language": lang
            }
        )
        
        num_labels = len(labels[lang])
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, problem_type="multi_label_classification")

        dataset = preprocess_dataset(ds[f'{lang}_train'])
        train_valalidation_split = dataset.train_test_split(test_size=0.2, seed=seed)

        train_dataset =  train_valalidation_split['train']
        validation_dataset = train_valalidation_split['test']

        training_args = TrainingArguments(
          output_dir=f'./results_{lang_mod_name}',
          eval_strategy="epoch",
          save_strategy="epoch",
          logging_dir=f'./logs_{lang_mod_name}',
          per_device_train_batch_size=bs,
          per_device_eval_batch_size=bs,
          num_train_epochs=epoch,
          weight_decay=wd,
          learning_rate=lr,
          logging_steps=10000,
          save_total_limit=2,
          load_best_model_at_end=True,
          metric_for_best_model="f1",
          seed=seed
        )

        trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset= validation_dataset,
          processing_class=tokenizer,
          compute_metrics=compute_metrics_wrapper(fin_mod_name),
        )

        trainer.train()
        metrics = trainer.evaluate()

        # print("Evaluation Metrics:", metrics)

        avg_runtime, avg_flops, total_flops, total_time = measure_runtime_and_flops(total_time, total_flops, trainer, validation_dataset)

        #store each model and succession score by individual language
        lang_mod_scores = scores[(scores['model'] == fin_mod_name) & (scores['lan'] == lang)]
        lang_avg_f1 = lang_mod_scores['f1'].mean()
        lang_model_stats[lang_mod_name] = round(score(lang_avg_f1, avg_runtime, avg_flops), 2)

        total_avg_runtime += avg_runtime
        total_avg_flops += avg_flops

        trainer.model.save_pretrained(f'./models/{lang_mod_name}')
        tokenizer.save_pretrained(f'./tokenizers/{lang_mod_name}')

      # store combined average succession score of all languages trained on. (Average Combined score of Java, Python, and Pharo)
      model_stats = scores[scores['model'] == fin_mod_name]
      fin_avg_f1 = model_stats['f1'].mean()
      final_model_stats[fin_mod_name] = round(score(fin_avg_f1, total_avg_runtime, total_avg_flops), 2)

------------------ Starting model ==> epochs: 5, batch size: 4, weights of decay: 0.01 ---------------------


c:\Users\ayoun\Miniconda3\envs\nbse2025\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 7614/7614 [00:00<00:00, 19594.18 examples/s]
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a differen

{'eval_loss': 0.14922671020030975, 'eval_f1': 0.5111289893021813, 'eval_precision': 0.5001785203458973, 'eval_recall': 0.5258611714970002, 'eval_runtime': 5.3812, 'eval_samples_per_second': 283.021, 'eval_steps_per_second': 70.802, 'epoch': 1.0}


                                                     
 40%|████      | 3046/7615 [03:21<04:40, 16.31it/s]

{'eval_loss': 0.10075070708990097, 'eval_f1': 0.7719487782969091, 'eval_precision': 0.7931749220157622, 'eval_recall': 0.7558382139560148, 'eval_runtime': 5.7295, 'eval_samples_per_second': 265.816, 'eval_steps_per_second': 66.498, 'epoch': 2.0}


...

100%|██████████| 2600/2600 [04:00<00:00, 13.40it/s]

{'eval_loss': 0.28284791111946106, 'eval_f1': 0.7055719873777272, 'eval_precision': 0.7996644760231717, 'eval_recall': 0.6599780930854929, 'eval_runtime': 0.6407, 'eval_samples_per_second': 405.792, 'eval_steps_per_second': 51.504, 'epoch': 20.0}


100%|██████████| 2600/2600 [04:02<00:00, 10.71it/s]


{'train_runtime': 242.774, 'train_samples_per_second': 85.512, 'train_steps_per_second': 10.71, 'train_loss': 0.06548944913423978, 'epoch': 20.0}


100%|██████████| 33/33 [00:00<00:00, 45.00it/s]


Evaluating the predictions.

In [23]:
from IPython.display import display

display(scores)

,model,lan,cat,precision,recall,f1
0,epoch-5_batchSize-4_weightsOfDecay-0.01,java,summary,0.900730,0.877667,0.889049
1,epoch-5_batchSize-4_weightsOfDecay-0.01,java,Ownership,0.964912,1.000000,0.982143
2,epoch-5_batchSize-4_weightsOfDecay-0.01,java,Expand,0.000000,0.000000,0.000000
3,epoch-5_batchSize-4_weightsOfDecay-0.01,java,usage,0.936019,0.903890,0.919674
4,epoch-5_batchSize-4_weightsOfDecay-0.01,java,Pointer,0.699588,0.899471,0.787037
...,...,...,...,...,...,...
2047,epoch-20_batchSize-8_weightsOfDecay-0.01,pharo,Responsibilities,0.673077,0.700000,0.686275
2048,epoch-20_batchSize-8_weightsOfDecay-0.01,pharo,Classreferences,0.750000,0.250000,0.375000
2049,epoch-20_batchSize-8_weightsOfDecay-0.01,pharo,Intent,0.937500,0.789474,0.857143
2050,epoch-20_batchSize-8_weightsOfDecay-0.01,pharo,Keymessages,0.703704,0.808511,0.752475


In [24]:
best_f1_scores = []
best_f1_java_row = scores[scores['lan'] == 'java'].loc[scores[scores['lan'] == 'java']['f1'].idxmax()]
best_f1_scores.append(best_f1_java_row)

best_f1_python_row = scores[scores['lan'] == 'python'].loc[scores[scores['lan'] == 'python']['f1'].idxmax()]
best_f1_scores.append(best_f1_python_row)

best_f1_pharo_row = scores[scores['lan'] == 'pharo'].loc[scores[scores['lan'] == 'pharo']['f1'].idxmax()]
best_f1_scores.append(best_f1_pharo_row)
best_f1_scores

[model        epoch-10_batchSize-4_weightsOfDecay-0.01
 lan                                              java
 cat                                         Ownership
 precision                                    0.982143
 recall                                            1.0
 f1                                           0.990991
 Name: 229, dtype: object,
 model        epoch-20_batchSize-8_weightsOfDecay-0.01
 lan                                            python
 cat                                        Parameters
 precision                                        0.85
 recall                                       0.910714
 f1                                            0.87931
 Name: 1861, dtype: object,
 model        epoch-20_batchSize-4_weightsOfDecay-0.01
 lan                                             pharo
 cat                                           Example
 precision                                    0.957447
 recall                                       0.891089
 f1       

In [18]:
lang_model_stats

{'java_epoch-5_batchSize-4_weightsOfDecay-0.01': 0.87,
 'python_epoch-5_batchSize-4_weightsOfDecay-0.01': 0.78,
 'pharo_epoch-5_batchSize-4_weightsOfDecay-0.01': 0.68,
 'java_epoch-5_batchSize-8_weightsOfDecay-0.01': 0.87,
 'python_epoch-5_batchSize-8_weightsOfDecay-0.01': 0.76,
 'pharo_epoch-5_batchSize-8_weightsOfDecay-0.01': 0.63,
 'java_epoch-10_batchSize-4_weightsOfDecay-0.01': 0.89,
 'python_epoch-10_batchSize-4_weightsOfDecay-0.01': 0.79,
 'pharo_epoch-10_batchSize-4_weightsOfDecay-0.01': 0.71,
 'java_epoch-10_batchSize-8_weightsOfDecay-0.01': 0.89,
 'python_epoch-10_batchSize-8_weightsOfDecay-0.01': 0.79,
 'pharo_epoch-10_batchSize-8_weightsOfDecay-0.01': 0.72,
 'java_epoch-15_batchSize-4_weightsOfDecay-0.01': 0.89,
 'python_epoch-15_batchSize-4_weightsOfDecay-0.01': 0.81,
 'pharo_epoch-15_batchSize-4_weightsOfDecay-0.01': 0.74,
 'java_epoch-15_batchSize-8_weightsOfDecay-0.01': 0.9,
 'python_epoch-15_batchSize-8_weightsOfDecay-0.01': 0.8,
 'pharo_epoch-15_batchSize-8_weightsOfD

In [19]:
def max_score_for_language(stats, lang):
    filtered_scores = {key: val for key, val in stats.items() if key.startswith(lang)}
    
    if filtered_scores:
        max_score = max(filtered_scores.values())
        return max_score
    else:
        return None  

best_lang_succ_scores = []

max_java_score = max_score_for_language(scores, 'java')
best_lang_succ_scores.append(max_java_score)

max_python_score = max_score_for_language(scores, 'python')
best_lang_succ_scores.append(max_python_score)

max_pharo_score = max_score_for_language(scores, 'pharo')
best_lang_succ_scores.append(max_pharo_score)

best_lang_succ_scores

[None, None, None]

In [20]:
final_model_stats

{'epoch-5_batchSize-4_weightsOfDecay-0.01': 0.75,
 'epoch-5_batchSize-8_weightsOfDecay-0.01': 0.72,
 'epoch-10_batchSize-4_weightsOfDecay-0.01': 0.77,
 'epoch-10_batchSize-8_weightsOfDecay-0.01': 0.77,
 'epoch-15_batchSize-4_weightsOfDecay-0.01': 0.79,
 'epoch-15_batchSize-8_weightsOfDecay-0.01': 0.79,
 'epoch-20_batchSize-4_weightsOfDecay-0.01': 0.79,
 'epoch-20_batchSize-8_weightsOfDecay-0.01': 0.8}

In [22]:
best_fin_succ_score = max(final_model_stats.values())
best_fin_succ_score

0.8